In [1]:
import hashlib
import random

# =======================
# SM2 椭圆曲线参数 (sm2p256v1)
# =======================
p = int("FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFF", 16)
a = int("FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFC", 16)
b = int("28E9FA9E9D9F5E344D5AEF20E93E3FBD6B5F6F4C52C9A7A3EB5C3C4F9DCC73E", 16)
n = int("FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFF7203DF6B21C6052B53BBF40939D54123", 16)
Gx = int("32C4AE2C1F1981195F9904466A39C9948FE30BBFF2660BE1715A4589334C74C7", 16)
Gy = int("BC3736A2F4F6779C59BDCEE36B692153D0A9877CC62A474002DF32E52139F0A0", 16)
G = (Gx, Gy)

# ========== 椭圆曲线运算 ==========
def inverse_mod(x, m):
    return pow(x, -1, m)

def point_add(P, Q):
    if P == (0, 0): return Q
    if Q == (0, 0): return P
    if P == Q:
        lam = (3 * P[0]*P[0] + a) * inverse_mod(2 * P[1], p) % p
    else:
        lam = (Q[1]-P[1]) * inverse_mod(Q[0]-P[0], p) % p
    x = (lam*lam - P[0] - Q[0]) % p
    y = (lam*(P[0]-x)-P[1]) % p
    return (x, y)

def scalar_mul(k, P):
    R = (0, 0)
    while k > 0:
        if k & 1:
            R = point_add(R, P)
        P = point_add(P, P)
        k >>= 1
    return R

# ========== 哈希函数 (模拟SM3) ==========
def sm3_hash(msg):
    return int(hashlib.sha256(msg.encode()).hexdigest(), 16)

# ========== SM2 KeyGen ==========
def sm2_keygen():
    d = random.randint(1, n-1)
    P = scalar_mul(d, G)
    return d, P

# ========== SM2 签名 ==========
def sm2_sign(msg, d, k=None):
    e = sm3_hash(msg)
    if not k:
        k = random.randint(1, n-1)
    x1, y1 = scalar_mul(k, G)
    r = (e + x1) % n
    s = (inverse_mod(1+d, n) * (k - r*d)) % n
    return (r, s, k)

# ========== SM2 验签 ==========
def sm2_verify(msg, sig, P):
    r, s = sig
    e = sm3_hash(msg)
    t = (r + s) % n
    x1, y1 = point_add(scalar_mul(s, G), scalar_mul(t, P))
    R = (e + x1) % n
    return R == r

# ========== 攻击：复用k恢复私钥 ==========
def attack_reuse_k(r1, s1, r2, s2):
    num = (s2 - s1) % n
    den = (s1 - s2 + r1 - r2) % n
    return (num * inverse_mod(den, n)) % n

# =========================
# 模拟攻击场景
# =========================
print("=== 1. Alice 生成密钥 ===")
dA, PA = sm2_keygen()
print(f"Alice 公钥: {PA}")
print(f"Alice 私钥 (仅Alice知道): {dA}")

# Alice 签两条消息，误用相同k
print("\n=== 2. Alice 使用相同k签名两条消息 ===")
k_fixed = random.randint(1, n-1)
m1 = "Alice first message"
m2 = "Alice second message"

r1, s1, _ = sm2_sign(m1, dA, k_fixed)
r2, s2, _ = sm2_sign(m2, dA, k_fixed)
print(f"签名1: r1={r1}, s1={s1}")
print(f"签名2: r2={r2}, s2={s2}")

# 攻击者恢复私钥
print("\n=== 3. 攻击者利用两次签名恢复私钥 ===")
dA_recovered = attack_reuse_k(r1, s1, r2, s2)
print(f"恢复的私钥: {dA_recovered}, 是否匹配: {dA == dA_recovered}")

# 伪造新消息签名
print("\n=== 4. 攻击者伪造签名 ===")
fake_msg = "This is a fake signed message"
r_fake, s_fake, _ = sm2_sign(fake_msg, dA_recovered)
print(f"伪造签名: r={r_fake}, s={s_fake}")
print("验证伪造签名:", sm2_verify(fake_msg, (r_fake, s_fake), PA))


=== 1. Alice 生成密钥 ===
Alice 公钥: (96182348615822097906329167833847028352674562863970427933536819529864515804010, 28495314961583272454618394348346193766539238364067201064786606742710001054779)
Alice 私钥 (仅Alice知道): 39740498730967353462529454594286702720299214613827093173805240373431192299443

=== 2. Alice 使用相同k签名两条消息 ===
签名1: r1=33184028689507590961614684689334421310288948716852851723351815883740495798186, s1=52415869675834309349379556937791648127504490868869987139738017148813056728545
签名2: r2=69599298667370672595909723771685394300663805493555760399949503838549899759742, s2=2132389044546292607315681150177433390523863199280412663351334152467103954344

=== 3. 攻击者利用两次签名恢复私钥 ===
恢复的私钥: 39740498730967353462529454594286702720299214613827093173805240373431192299443, 是否匹配: True

=== 4. 攻击者伪造签名 ===
伪造签名: r=57389017497857855863459954365547949119389426713671528339822643360222561906097, s=72667335261518464473567786836763345042951459373351988650575511094654016558435
验证伪造签名: True
